# Baseline script of San Francisco Crime Classification

Baseline script. Hope this helps yours.

In [1]:
import numpy as np
import pandas as pd

## Load Data

In [2]:
train = pd.read_pickle("../all_df.p")
train.head(3)

,날짜,순위,영화명,개봉일,매출액,매출액 점유율,매출액증감 (전일대비),매출액증감율 (전일대비),누적매출액,관객수,...,스크린수,상영횟수,대표국적,국적,제작사,배급사,등급,장르,감독,배우
4,2014년 08월 07일(목),1,명량,2014-07-30 00:00:00,4892511700,0.595,-380933700,-0.1,61033902816,652776,...,1239,6776,한국,한국,(주)빅스톤픽쳐스,씨제이이앤엠(주),15세이상관람가,"사극,액션",김한민,"최민식,류승룡,조진웅,김명곤,진구,이정현,권율,노민우,김태훈,이승준,오타니 료헤이,..."
5,2014년 08월 07일(목),2,해적: 바다로 간 산적,2014-08-06 00:00:00,1890503200,0.23,-130889400,-0.1,4137374300,254862,...,795,3932,한국,한국,(주)하리마오픽쳐스,롯데쇼핑㈜롯데엔터테인먼트,12세이상관람가,어드벤처,이석훈,"김남길,손예진,유해진,이경영,오달수,김태우,박철민,신정근,김원해,조달환,조희봉,정성..."
6,2014년 08월 07일(목),3,드래곤 길들이기 2,2014-07-23 00:00:00,437352300,0.053,-27892900,-0.1,19775433889,58182,...,423,1016,미국,미국,드림웍스 애니메이션,씨제이이앤엠(주),전체관람가,"애니메이션,액션,어드벤처",딘 데블로이스,"제이 바루첼,제라드 버틀러,케이트 블란쳇,아메리카 페레라"


In [3]:
train.to_csv('csvfile.csv', index=False)

In [4]:
def classify(row) :
    pg = row['등급']
    if pg.find('15') != -1 :
        pg = '15'
    elif pg.find('12') != -1 :
        pg ='12'
    elif pg.find('19') != -1  :
        pg = '19'
    else :
        pg = '전체'
    row['등급'] = pg
    return row

In [5]:
train['등급'] = train['등급'].fillna('전체')

In [6]:
train = train.apply(classify, axis=1)

In [7]:
name = train[train["누적관객수"] >= 100000]['영화명']

name = name.unique()

train = train.loc[train['영화명'].isin(name)]

In [8]:
import re

train["날짜"] = train["날짜"].apply(lambda date: re.sub('[(년월화수목금토일)]', '', date))
train["날짜"] = train["날짜"].apply(lambda date: date.replace(" ", "-"))
train["날짜"].head(3)

train["개봉일"] = pd.to_datetime(train["개봉일"], errors='coerce')
train["개봉일"].head()

train["상영기간"] = pd.to_datetime(train["날짜"]) - pd.to_datetime(train["개봉일"])
train = train.dropna(subset=['상영기간'])

train = train[train["상영기간"] >= '0 days']
train = train[train["상영기간"] < '30 days']

In [9]:
test = pd.read_pickle("../test.p")

In [10]:
test

,날짜,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,2016년 09월 30일(금),"고산자, 대동여지도",2016-09-07,한국,한국,(주)시네마서비스,씨제이이앤엠(주),전체관람가,사극,강우석,"차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수"
1,2016년 09월 30일(금),매그니피센트 7,2016-09-14,미국,미국,,유니버설픽쳐스인터내셔널 코리아(유),15세이상관람가,"액션, 서부극(웨스턴)",안톤 후쿠아,"덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,..."
2,2016년 09월 30일(금),카페 소사이어티,2016-09-14,미국,미국,,CGV아트하우스,전체관람가,"멜로/로맨스, 드라마",우디 앨런,"제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리"


In [11]:
test = test.apply(classify,axis=1)

In [12]:
vector_columns = test.columns.drop(['날짜', '영화명', '개봉일'])

In [13]:
vector_columns

Index(['대표국적', '국적', '제작사', '배급사', '등급', '장르', '감독', '배우'], dtype='object')

In [14]:
d_list = []
for each in train.iterrows():
    row = ""
    for column in vector_columns.values :
        if type(each[1][column]) == str :
            row = row + each[1][column] + " "
    d_list.append(row)

In [15]:
y_list = train["누적관객수"].values

In [16]:
test_d_list = []
for each in test.iterrows():
    row = ""
    for column in vector_columns.values :
        if type(each[1][column]) == str :
            row = row + each[1][column] + " "
    test_d_list.append(row)

In [17]:
test_d_list

['한국 한국 (주)시네마서비스 씨제이이앤엠(주) 전체 사극 강우석 차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수 ',
 '미국 미국  유니버설픽쳐스인터내셔널 코리아(유) 15 액션, 서부극(웨스턴) 안톤 후쿠아 덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,캠 지갠뎃 ',
 '미국 미국  CGV아트하우스 전체 멜로/로맨스, 드라마 우디 앨런 제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리 ']

## Vectorize


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
vectorizer = TfidfVectorizer()
x_list = vectorizer.fit_transform(d_list)

In [20]:
x_list

<7726x2570 sparse matrix of type '<class 'numpy.float64'>'
	with 153376 stored elements in Compressed Sparse Row format>

In [21]:
x_list.shape

(7726, 2570)

In [22]:
x_list._shape = (x_list.shape[0], x_list.shape[1] + 1)

In [23]:
train["상영기간"] = train["상영기간"].astype('timedelta64[D]').astype(int)

In [24]:
train.iloc[0]['상영기간']

8

In [25]:
x_list.shape[0]

7726

In [26]:
train.shape

(7726, 24)

In [27]:
x_list.shape

(7726, 2571)

In [28]:
days = train['상영기간'].values

In [29]:
for i in range(x_list.shape[0]) :
    x_list[i, x_list.shape[1] - 1] = days[i]

/Users/mac/anaconda/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


## CV

In [30]:

from sklearn.tree import DecisionTreeRegressor
model_rfr = DecisionTreeRegressor()
model_rfr.fit(x_list, y_list)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, presort=False, random_state=None,
           splitter='best')

In [31]:

from sklearn.ensemble import RandomForestRegressor
model_rfr = RandomForestRegressor(n_estimators=100, n_jobs=-1)
model_rfr.fit(x_list, y_list)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=-1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [32]:
test

,날짜,영화명,개봉일,대표국적,국적,제작사,배급사,등급,장르,감독,배우
0,2016년 09월 30일(금),"고산자, 대동여지도",2016-09-07,한국,한국,(주)시네마서비스,씨제이이앤엠(주),전체,사극,강우석,"차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수"
1,2016년 09월 30일(금),매그니피센트 7,2016-09-14,미국,미국,,유니버설픽쳐스인터내셔널 코리아(유),15,"액션, 서부극(웨스턴)",안톤 후쿠아,"덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,..."
2,2016년 09월 30일(금),카페 소사이어티,2016-09-14,미국,미국,,CGV아트하우스,전체,"멜로/로맨스, 드라마",우디 앨런,"제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리"


In [33]:
vectorizer.vocabulary_['씨제이이앤엠']

1140

In [34]:
test_d_list

['한국 한국 (주)시네마서비스 씨제이이앤엠(주) 전체 사극 강우석 차승원,유준상,김인권,남지현,신동미,남경읍,태인호,공형진,김종수 ',
 '미국 미국  유니버설픽쳐스인터내셔널 코리아(유) 15 액션, 서부극(웨스턴) 안톤 후쿠아 덴젤 워싱턴,크리스 프랫,이병헌,에단 호크,맷 보머,빈센트 도노프리오,해일리 베넷,캠 지갠뎃 ',
 '미국 미국  CGV아트하우스 전체 멜로/로맨스, 드라마 우디 앨런 제시 아이젠버그, 크리스틴 스튜어트, 스티브 카렐, 블레이크 라이블리 ']

In [35]:
test_x_list = vectorizer.transform(test_d_list)

In [36]:
vectorizer.vocabulary_

{'크로우': 2259,
 '이광호': 1536,
 '문채원': 668,
 '정현석': 1940,
 '김나영': 139,
 '조희성': 2034,
 '류승룡': 550,
 '토니': 2318,
 '워커': 1410,
 '스마일이엔티': 1050,
 '박성규': 724,
 '명필름': 642,
 '포너': 2378,
 '유지태': 1477,
 'sf': 4,
 '사무엘': 916,
 '송용호': 1028,
 '이광훈': 1537,
 '김대환': 148,
 '카터': 2190,
 '박주희': 770,
 '히다카': 2566,
 '모니카': 645,
 '장미진': 1818,
 '김희찬': 345,
 '정만식': 1883,
 '코리아스크린': 2220,
 '월광': 1411,
 '김병춘': 180,
 '조연우': 2005,
 '이승준': 1637,
 '임지연': 1803,
 '김혜자': 334,
 '윤여진': 1503,
 '맥키넌': 624,
 '메리': 635,
 'new': 3,
 '베이': 849,
 '안상완': 1174,
 '스즈코': 1056,
 '매튜': 612,
 '신나라': 1100,
 '유성진': 1454,
 '송승헌': 1025,
 '이승호': 1641,
 '지페르만': 2071,
 '권병길': 82,
 '케이티스': 2209,
 '윌리엄스': 1439,
 '이주현': 1730,
 '폴슨': 2390,
 '마사미': 590,
 '던칸': 423,
 '다이안': 406,
 '이학주': 1760,
 '박민철': 712,
 '윌리엄슨': 1440,
 '에반스': 1272,
 '카니': 2177,
 '양대성': 1234,
 '헤니': 2487,
 '이동민': 1554,
 '이용출': 1668,
 '캘링': 2198,
 '안소니': 1180,
 '우정국': 1398,
 '전혜진': 1868,
 '필로': 2423,
 '이일형': 1699,
 '이연두': 1653,
 '밀러': 691,
 '성유빈': 980,
 '장태성': 1837,
 '콜트레인': 2242,

In [37]:
vectorizer.transform(['유준상'])

<1x2570 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [38]:
test_x_list._shape = (3, 2571)

In [39]:
import re

test["날짜"] = test["날짜"].apply(lambda date: re.sub('[(년월화수목금토일)]', '', date))
test["날짜"] = test["날짜"].apply(lambda date: date.replace(" ", "-"))
test["날짜"].head(3)

test["개봉일"] = pd.to_datetime(test["개봉일"], errors='coerce')
test["개봉일"].head()

test["상영기간"] = pd.to_datetime(test["날짜"]) - pd.to_datetime(test["개봉일"])
test = test.dropna(subset=['상영기간'])

test = test[test["상영기간"] >= '0 days']
test = test[test["상영기간"] < '30 days']
test["상영기간"] = test["상영기간"].astype('timedelta64[D]').astype(int)
test_days = test['상영기간'].values

In [41]:
test_days = [1,1,1]

In [42]:
for i in range(test_x_list.shape[0]) :
    test_x_list[i, test_x_list.shape[1] - 1] = test_days[i]

/Users/mac/anaconda/lib/python3.5/site-packages/scipy/sparse/compressed.py:730: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [43]:
model_rfr.predict(test_x_list)

array([  84264.83,  171622.58,  100565.81])